# 1. Install and Import Dependencies

In [ ]:
!pip install mediapipe opencv-python

In [ ]:
import mediapipe as mp
import cv2
import numpy as np
from numpy import interp
import uuid
import os
from typing import Mapping, Tuple
from mediapipe.python.solutions import drawing_styles

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# 2. Draw Hands
<img src=https://i.imgur.com/qpRACer.png />

In [ ]:
def getColor(zDist):
    print(zDist)
    c = int(interp(zDist, [0,15], [0,255]))
    return (c,c,c)

In [ ]:
def createLandMarks(hand_landmarks): #-> Mapping[int, mp_drawing.DrawingSpec]:
  hand_landmark_style = {}  
  for k, v in drawing_styles._HAND_LANDMARK_STYLE.items():
    for landmark in k:
      c = getColor(abs(hand_landmarks.landmark[landmark].z*100))
      r = int(abs(hand_landmarks.landmark[landmark].z*100))
      hand_landmark_style[landmark] =   mp_drawing.DrawingSpec(color=c, thickness=drawing_styles._THICKNESS_DOT, circle_radius= r )
  return hand_landmark_style        

In [ ]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(static_image_mode=False,min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
        ret, frame = cap.read()
        
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
#         print('Handedness:', results.multi_handedness)

        #Get image H ,W
        image_height, image_width, _ = image.shape
        
        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand_landmarks  in enumerate(results.multi_hand_landmarks):
#                 print('hand_landmarks:', hand_landmarks)
                print(
                    f'Index finger tip coordinates: (',
                    f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
                    f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height}) '
                    f'{abs(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].z*100)})'
                )
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS, 
#                                         mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
#                                         mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),)
                                      createLandMarks(hand_landmarks),
                                      mp_drawing_styles.get_default_hand_connections_style())
                    
        plot = np.zeros([image_height, image_width, 3], dtype=np.uint8)                
        if results.multi_hand_world_landmarks:
            for num,hand_world_landmarks in enumerate(results.multi_hand_world_landmarks):                
                for idx,landMrk in enumerate(hand_world_landmarks.landmark):
                    hand_world_landmarks.landmark[idx].x += 0.5
                    hand_world_landmarks.landmark[idx].y += 0.5
                mp_drawing.draw_landmarks(plot,hand_world_landmarks, mp_hands.HAND_CONNECTIONS)
#                 mp_drawing.plot_landmarks(hand_world_landmarks, mp_hands.HAND_CONNECTIONS, azimuth=5)
        
        cv2.imshow('Plot', plot)
        cv2.imshow('Hand Tracking', image)        

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()